In [2]:
# Install necessary libraries
!pip install python-aiml pandas requests

In [3]:
# Import Libraries
import aiml
import os
import json
import pandas as pd
import requests

In [5]:
# Create AIML File (medical.aiml)
aiml_content = """
<aiml version="1.0.1" encoding="UTF-8">

<!-- Greeting -->
<category>
    <pattern>HELLO</pattern>
    <template>Hello! I am your Medical Chatbot. Tell me your symptoms, and I will try to help.</template>
</category>

<!-- Fallback -->
<category>
    <pattern>*</pattern>
    <template>Sorry, I am not sure about that symptom. Can you describe it differently?</template>
</category>

</aiml>
"""

with open("medical.aiml", "w") as f:
    f.write(aiml_content)

print("medical.aiml created successfully!")


medical.aiml created successfully!


In [7]:
# Create JSON dataset for 10 diseases and symptoms
json_data = {
    "fever": {"diseases": ["Flu", "COVID-19"], "precautions": ["Rest", "Drink water", "Monitor temperature", "Consult doctor"]},
    "headache": {"diseases": ["Migraine", "Stress"], "precautions": ["Rest", "Hydrate", "Avoid screen strain"]},
    "cough": {"diseases": ["Common Cold", "COVID-19"], "precautions": ["Stay hydrated", "Humidifier", "Consult doctor if persistent"]},
    "stomach pain": {"diseases": ["Indigestion", "Gastritis"], "precautions": ["Eat light meals", "Avoid spicy food", "Consult doctor if severe"]},
    "fatigue": {"diseases": ["Anemia", "Thyroid disorder"], "precautions": ["Eat iron-rich foods", "Rest", "Consult doctor for tests"]},
    "sore throat": {"diseases": ["Tonsillitis", "Common Cold"], "precautions": ["Stay hydrated", "Gargle warm salt water", "Consult doctor if persistent"]},
    "shortness of breath": {"diseases": ["Asthma", "COVID-19"], "precautions": ["Use inhaler if prescribed", "Consult doctor immediately"]},
    "rash": {"diseases": ["Allergy", "Chickenpox"], "precautions": ["Avoid allergen", "Use prescribed creams"]},
    "nausea": {"diseases": ["Food Poisoning", "Pregnancy"], "precautions": ["Stay hydrated", "Eat light meals", "Consult doctor"]},
    "joint pain": {"diseases": ["Arthritis", "Injury"], "precautions": ["Rest", "Use pain relief medication", "Consult doctor"]}
}

with open("diseases.json", "w") as f:
    json.dump(json_data, f, indent=4)

print("diseases.json created successfully with 10 diseases!")



diseases.json created successfully with 10 diseases!


In [8]:
# Load JSON data and create Pandas DataFrame
with open("diseases.json", "r") as f:
    diseases_data = json.load(f)

df = pd.DataFrame([
    {'Symptom': symptom.capitalize(), 'Diseases': ', '.join(info['diseases']), 'Precautions': ', '.join(info['precautions'])}
    for symptom, info in diseases_data.items()
])

print("Current Symptom-Disease Dataset:")
df



Current Symptom-Disease Dataset:


,Symptom,Diseases,Precautions
0,Fever,"Flu, COVID-19","Rest, Drink water, Monitor temperature, Consul..."
1,Headache,"Migraine, Stress","Rest, Hydrate, Avoid screen strain"
2,Cough,"Common Cold, COVID-19","Stay hydrated, Humidifier, Consult doctor if p..."
3,Stomach pain,"Indigestion, Gastritis","Eat light meals, Avoid spicy food, Consult doc..."
4,Fatigue,"Anemia, Thyroid disorder","Eat iron-rich foods, Rest, Consult doctor for ..."
5,Sore throat,"Tonsillitis, Common Cold","Stay hydrated, Gargle warm salt water, Consult..."
6,Shortness of breath,"Asthma, COVID-19","Use inhaler if prescribed, Consult doctor imme..."
7,Rash,"Allergy, Chickenpox","Avoid allergen, Use prescribed creams"
8,Nausea,"Food Poisoning, Pregnancy","Stay hydrated, Eat light meals, Consult doctor"
9,Joint pain,"Arthritis, Injury","Rest, Use pain relief medication, Consult doctor"


In [11]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

https://docs.google.com/spreadsheets/d/1PVX2D_LzNu-XInsSLYOAzCZLxfiRsGUhIMUVUEqtqGU/edit#gid=0


In [13]:
# Create AIML kernel
kernel = aiml.Kernel()
kernel.learn("medical.aiml")
print("AIML kernel loaded successfully!")

Loading medical.aiml...done (0.00 seconds)
AIML kernel loaded successfully!


In [15]:
# Function to suggest diseases based on partial symptom input using Pandas

def query_disease_dynamic(user_input):
    user_input = user_input.lower().strip()
    matches = df[df['Symptom'].str.contains(user_input, case=False)]
    if not matches.empty:
        for _, row in matches.iterrows():
            print(f"Symptom: {row['Symptom']}")
            print(f"Possible Diseases: {row['Diseases']}")
            print(f"Precautions: {row['Precautions']}")
    else:
        print("No data found for this symptom in JSON dataset.")

In [16]:
# Start Chatbot (AIML + dynamic Pandas query)
print("Welcome to the Medical Chatbot (AIML + Dynamic Dataset)!")
print("Type 'quit' to exit.")

while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        print("Chatbot: Goodbye! Stay healthy.")
        break
    # Respond using AIML (general fallback/greeting)
    aiml_response = kernel.respond(user_input.upper())
    if aiml_response:
        print("Chatbot (AIML):", aiml_response)
    # Use dynamic Pandas query for symptom-based suggestions
    query_disease_dynamic(user_input)


Welcome to the Medical Chatbot (AIML + Dynamic Dataset)!
Type 'quit' to exit.
You: fever
Chatbot (AIML): Sorry, I am not sure about that symptom. Can you describe it differently?
Symptom: Fever
Possible Diseases: Flu, COVID-19
Precautions: Rest, Drink water, Monitor temperature, Consult doctor
You: quit
Chatbot: Goodbye! Stay healthy.
